In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8041"


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Bob"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}


def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener, message_listener], defaults=True)

In [3]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State invitation
Connection ID 10bf00c7-fe28-4f15-ac2c-f598e3e60257
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '31d22c96-b531-467a-8637-867f0e9199ac', 'label': 'Health Researcher', 'serviceEndpoint': 'http://172.17.0.1:8040', 'recipientKeys': ['Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE']}
Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State request


## Copy the invitation output from 4 and to the hospital notebook you are connecting to.

In [4]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State response
ACCEPT REQUEST
{'created_at': '2021-01-23 14:21:34.771903Z', 'routing_state': 'none', 'accept': 'manual', 'invitation_key': 'Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE', 'initiator': 'self', 'my_did': '8wvYweJAFRYr4LZA77hYqa', 'invitation_mode': 'once', 'updated_at': '2021-01-23 14:22:26.279182Z', 'connection_id': '10bf00c7-fe28-4f15-ac2c-f598e3e60257', 'their_label': 'Royal Infirmary of Edinburgh', 'state': 'response', 'their_did': 'ALrPXCqEuqa5wYFUDDkfeo'}
state response


In [5]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': 'fed59ad7-f56e-4724-832d-00a9234dfa6b'}
Connection Handler Called
Connection 10bf00c7-fe28-4f15-ac2c-f598e3e60257 in State active


In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")

In [6]:
# Data pre-processing

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# prep
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler
from sklearn import metrics

print("COORDINATOR IS CLEANING THE VALIDATION SET")

#Read in Data
train_df = pd.read_csv('coordinator_validation_data.csv')

print("COORDINATOR DATA", train_df)

########## START DATA CLEANING ###############
#Let’s get rid of the variables "Timestamp",“comments”, “state” just to make our lives easier.
train_df = train_df.drop(['comments'], axis= 1)
train_df = train_df.drop(['state'], axis= 1)
train_df = train_df.drop(['Timestamp'], axis= 1)

# Assign default values for each data type
defaultInt = 0
defaultString = 'NaN'
defaultFloat = 0.0

# Create lists by data tpe
intFeatures = ['Age']
stringFeatures = ['Gender', 'Country', 'self_employed', 'family_history', 'treatment', 'work_interfere',
                 'no_employees', 'remote_work', 'tech_company', 'anonymity', 'leave', 'mental_health_consequence',
                 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
                 'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 'wellness_program',
                 'seek_help']
floatFeatures = []

# Clean the NaN's
for feature in train_df:
    if feature in intFeatures:
        train_df[feature] = train_df[feature].fillna(defaultInt)
    elif feature in stringFeatures:
        train_df[feature] = train_df[feature].fillna(defaultString)
    elif feature in floatFeatures:
        train_df[feature] = train_df[feature].fillna(defaultFloat)
    else:
        print('Error: Feature %s not recognized.' % feature)

#clean 'Gender'
#Slower case all columm's elements
gender = train_df['Gender'].str.lower()
#print(gender)

#Select unique elements
gender = train_df['Gender'].unique()

#Made gender groups
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for (row, col) in train_df.iterrows():

    if str.lower(col.Gender) in male_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

#Get rid of bullshit
stk_list = ['A little about you', 'p']
train_df = train_df[~train_df['Gender'].isin(stk_list)]

#complete missing age with mean
train_df['Age'].fillna(train_df['Age'].median(), inplace = True)

# Fill with media() values < 18 and > 120
s = pd.Series(train_df['Age'])
s[s<18] = train_df['Age'].median()
train_df['Age'] = s
s = pd.Series(train_df['Age'])
s[s>120] = train_df['Age'].median()
train_df['Age'] = s

#Ranges of Age
train_df['age_range'] = pd.cut(train_df['Age'], [0,20,30,65,100], labels=["0-20", "21-30", "31-65", "66-100"], include_lowest=True)

#There are only 0.20% of self work_interfere so let's change NaN to "Don't know
#Replace "NaN" string from defaultString

train_df['work_interfere'] = train_df['work_interfere'].replace([defaultString], 'Don\'t know' )

#Encoding data
labelDict = {}
for feature in train_df:
    le = preprocessing.LabelEncoder()
    le.fit(train_df[feature])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    train_df[feature] = le.transform(train_df[feature])
    # Get labels
    labelKey = 'label_' + feature
    labelValue = [*le_name_mapping]
    labelDict[labelKey] =labelValue

#Get rid of 'Country'
train_df = train_df.drop(['Country'], axis= 1)

# Scaling Age
scaler = MinMaxScaler()
train_df['Age'] = scaler.fit_transform(train_df[['Age']])

# define X and y
feature_cols = ['Age', 'Gender', 'family_history', 'benefits', 'care_options', 'anonymity', 'leave', 'work_interfere']
X = train_df[feature_cols]
y = train_df.treatment

# split X and y into training and testing sets
X_test, y_test = X, y

# Transform pandas dataframe to torch tensor for DL

x_test_data = torch.from_numpy(X_test.values)
x_test_data = x_test_data.float()

y_test_data = []
for data in y_test.values:
    y_test_data.append([data])
y_test_data = torch.tensor(y_test_data).float()

print("VALIDATION SET HAS BEEN CLEANED")

########## END DATA CLEANING ###############


COORDINATOR IS CLEANING THE VALIDATION SET
COORDINATOR DATA                Timestamp  Age  Gender         Country state self_employed  \
0    2014-08-29 09:29:37   25    Male   United States    PA            No   
1    2014-08-29 09:31:37   42    male   United States    IN            No   
2    2014-08-29 09:31:49   34    male   United States    PA            No   
3    2014-08-29 09:33:43   26  female   United States    OH            No   
4    2014-08-29 09:35:46   35    Male  United Kingdom   NaN            No   
..                   ...  ...     ...             ...   ...           ...   
254  2015-09-12 11:17:21   26    male  United Kingdom   NaN            No   
255  2015-09-26 01:07:35   32    Male   United States    IL            No   
256  2015-11-07 12:36:58   34    male   United States    CA            No   
257  2015-11-30 21:25:06   46       f   United States    NC            No   
258  2016-02-01 23:04:31   25    Male   United States    IL            No   

    family_hist

In [8]:
import torch

# models
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
import sys

from opacus import PrivacyEngine

# The Researcher generates the model

model = nn.Sequential(
            nn.Linear(8, 4),
            nn.Sigmoid(),
            nn.Linear(4, 2),
            nn.Sigmoid(),
            nn.Linear(2, 1),
            nn.Sigmoid()
)

torch.save(model, "../model.pt")


In [ ]:
# Convert the model.pt to Base64 text and Send it to the hospital




In [12]:
# Receive the Trained model.pt from the hospital and validate it

# Pull in model

model_dir = os.getcwd() + "/../trained_model.pt"

model = torch.load(model_dir)

print("HOSPITAL MODEL LOADED")
print("\nPRINTING PARAMETERS:\n\n")


print(model)

print('\n')

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

# Validation Logic
print("\n\n\nHOSPITAL IS VALIDATING")

#BINARIZE PREDICTION FOR CONFUSION MATRIX

pred = []

for data in  model(x_test_data):
    if data > .5:
        pred.append(1)
    else:
        pred.append(0)


confusion = metrics.confusion_matrix(pred, y_test_data)

print("Model loss on validation set: ", (model(x_test_data) - y_test_data).sum())
print("Confusion Matrix:\n                Actual_True, Actual_False \n Predicted_True    ",confusion[1][1],"   |     ",confusion[1][0],"    \n Predicted_False   ",confusion[0][1],"     |      ",confusion[0][0],"    \n")



HOSPITAL MODEL LOADED

PRINTING PARAMETERS:


Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)


0.weight tensor([[ 1.5149,  1.4357, -0.7174,  0.3806,  0.3705,  1.1451, -1.0300, -4.0701],
        [-2.1123,  1.6906, -1.8210, -0.3734, -0.5095, -1.0946, -1.4837,  1.5619],
        [-0.2669,  0.9192, -2.6004, -0.3604, -0.9123, -0.4561,  1.5589, -0.5680],
        [-0.5884,  0.8660,  0.6667, -1.6752,  0.4187,  0.8137,  1.2785, -2.8669]])
0.bias tensor([2.0214, 1.1634, 0.6690, 0.6215])
2.weight tensor([[ 4.0814,  2.8046,  1.8664,  2.8660],
        [-2.4376, -1.6406, -0.7205, -2.2948]])
2.bias tensor([-2.9719,  1.7671])
4.weight tensor([[-6.6559,  4.7370]])
4.bias tensor([2.0326])



HOSPITAL IS VALIDATING
Model loss on validation set:  tensor(-5.6757, grad_fn=<SumBackward0>)
Confusion Matrix:
                Actu

In [ ]:
# Send the model to the next Hospital
# Receive the Trained Model and Validate again
# Repeat until all Hospitals Trained the model

In [ ]:
response = await agent_controller.terminate()
print(response)